In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import csv
data={}
test_data = {}
i=0
curr = ''
with open('quartileData\\taskData_staiTop.csv', 'r' ) as theFile:
    reader = csv.reader(theFile)
    headers = next(reader, None)
    for line in reader:
        if line[70] != curr:
            if i > 0:
                block = [
                    {
                        'action': np.array([actions])-1,
                        'state': np.array([states]),
                        'reward': np.array([rewards]),
                        'id': 'S'+str(i),

                        'block': 0
                    }

                ]
                if i<226:
                    data['S'+str(i)] = block
                else:
                    test_data['S'+str(i)] = block
            curr = line[70]
            i += 1
            actions = []
            states = []
            rewards = []
        actions.append(int(float(line[8])))
        states.append([int(float(line[3])),int(float(line[4]))])
        rewards.append(int(float(line[10])))

In [3]:
data

{'S1': [{'action': array([[0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1,
           1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
           0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1,
           0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1,
           1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
           1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
           1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
           0, 0, 1, 1, 0]]),
   'state': array([[[ 4,  7],
           [ 7,  8],
           [ 8,  7],
           [ 7,  4],
           [ 8,  4],
           [ 7,  8],
           [ 4,  7],
           [ 4,  7],
           [ 8,  7],
           [ 7,  4],
           [ 8,  7],
           [ 4,  8],
           [ 7,  8],
           [ 4,  8],
           [ 8,  4],
           [ 4,  8],
           [ 9,  4],
           [ 4,  9],
           [ 9,  4],
        

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=2, s_size=2, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/taskData_staiTop/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-08-04 18:23:07,194 - DeepRL - DEBUG - version control: (None, None)
2021-08-04 18:23:07,194 - DeepRL - DEBUG - learning rate: 0.01
2021-08-04 18:23:07,195 - DeepRL - DEBUG - global iters: 50
2021-08-04 18:23:07,196 - DeepRL - DEBUG - training data-points: 225
2021-08-04 18:23:07,196 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-08-04 18:23:07,642 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/taskData_staiTop/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-04 18:23:22,675 - DeepRL - DEBUG - global iter =    0 total obj: 24801.0994
2021-08-04 18:23:36,615 - DeepRL - DEBUG - global iter =    1 total obj: 24757.8406
2021-08-04 18:23:50,744 - DeepRL - DEBUG - global iter =    2 total obj: 24508.3516
2021-08-04 18:24:04,849 - DeepRL - DEBUG - global iter =    3 total obj: 23905.2164
2021-08-04 18:24:19,153 - DeepRL - DEBUG - global iter =    4 total obj: 23223.6259
2021-08-04 18:24:33,190 - DeepRL - DEBUG - global iter =    5 total obj: 22814.1220
2021-08-04 18:24:47,264 - DeepRL - DEBUG - global iter =    6 total obj: 22539.2414
2021-08-04 18:25:01,229 - DeepRL - DEBUG - global iter =    7 total obj: 22331.8777
2021-08-04 18:25:15,236 - DeepRL - DEBUG - global iter =    8 total obj: 22195.3292
2021-08-04 18:25:29,222 - DeepRL - DEBUG - global iter =    9 total obj: 22093.5669
2021-08-04 18:25:43,196 - DeepRL - DEBUG - global iter =   10 total obj: 21997.3139
2021-08-04 18:25:56,870 - DeepRL - DEBUG - global iter =   11 total obj: 218

INFO:tensorflow:../results/taskData_staiTop/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
from actionflow.data.data_process import DataProcess
train_merged = DataProcess.merge_data(data)
train_merged

{'merged': [{'reward': array([[ 1,  1,  1, ...,  0,  1,  0],
          [ 0,  1,  1, ..., -1, -1, -1],
          [ 1,  1,  1, ...,  1,  0, -1],
          ...,
          [ 0,  1,  1, ..., -1, -1, -1],
          [ 0,  0,  1, ...,  0,  0,  1],
          [ 0,  1,  1, ...,  0,  1, -1]]),
   'action': array([[ 0,  1,  0, ...,  0,  0,  0],
          [ 1,  0,  0, ..., -1, -1, -1],
          [ 0,  0,  0, ...,  0,  1, -1],
          ...,
          [ 1,  0,  0, ..., -1, -1, -1],
          [ 0,  1,  1, ...,  1,  1,  1],
          [ 0,  1,  0, ...,  1,  0, -1]]),
   'state': array([[[ 9,  2],
           [ 2,  9],
           [ 9, 12],
           ...,
           [11, 10],
           [ 9, 10],
           [11, 10]],
   
          [[ 1,  2],
           [ 1,  2],
           [ 1,  8],
           ...,
           [-1, -1],
           [-1, -1],
           [-1, -1]],
   
          [[ 2,  4],
           [ 2,  4],
           [ 2,  4],
           ...,
           [ 5,  8],
           [ 9,  8],
           [-1, -1]]

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [7]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-08-04 18:34:14,570 - DeepRL - DEBUG - version control: (None, None)
2021-08-04 18:34:14,571 - DeepRL - DEBUG - learning rate: 0.01
2021-08-04 18:34:14,572 - DeepRL - DEBUG - global iters: 50
2021-08-04 18:34:14,572 - DeepRL - DEBUG - training data-points: 225
2021-08-04 18:34:14,572 - DeepRL - DEBUG - test data-points: 32
2021-08-04 18:34:14,963 - DeepRL - DEBUG - opt started...
2021-08-04 18:34:14,964 - DeepRL - DEBUG - started testing...
2021-08-04 18:34:19,236 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/taskData_staiTop/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-04 18:34:33,170 - DeepRL - DEBUG - global iter =    0 total obj: 24784.5005
2021-08-04 18:34:46,333 - DeepRL - DEBUG - global iter =    1 total obj: 24379.5677
2021-08-04 18:34:59,767 - DeepRL - DEBUG - global iter =    2 total obj: 23480.1375
2021-08-04 18:35:13,215 - DeepRL - DEBUG - global iter =    3 total obj: 22913.7685
2021-08-04 18:35:26,649 - DeepRL - DEBUG - global iter =    4 total obj: 22548.8208
2021-08-04 18:35:40,077 - DeepRL - DEBUG - global iter =    5 total obj: 22320.6534
2021-08-04 18:35:53,552 - DeepRL - DEBUG - global iter =    6 total obj: 22122.3479
2021-08-04 18:36:06,989 - DeepRL - DEBUG - global iter =    7 total obj: 21958.6572
2021-08-04 18:36:20,422 - DeepRL - DEBUG - global iter =    8 total obj: 21835.0443
2021-08-04 18:36:33,869 - DeepRL - DEBUG - global iter =    9 total obj: 21718.6380
2021-08-04 18:36:33,871 - DeepRL - DEBUG - started testing...
2021-08-04 18:36:38,460 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/taskData_staiTop/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-04 18:36:52,546 - DeepRL - DEBUG - global iter =   10 total obj: 21614.2547
2021-08-04 18:37:06,006 - DeepRL - DEBUG - global iter =   11 total obj: 21518.8285
2021-08-04 18:37:19,472 - DeepRL - DEBUG - global iter =   12 total obj: 21448.8100
2021-08-04 18:37:32,940 - DeepRL - DEBUG - global iter =   13 total obj: 21385.4732
2021-08-04 18:37:46,389 - DeepRL - DEBUG - global iter =   14 total obj: 21344.0617
2021-08-04 18:37:59,860 - DeepRL - DEBUG - global iter =   15 total obj: 21276.5018
2021-08-04 18:38:13,341 - DeepRL - DEBUG - global iter =   16 total obj: 21229.9548
2021-08-04 18:38:26,805 - DeepRL - DEBUG - global iter =   17 total obj: 21185.0542
2021-08-04 18:38:40,284 - DeepRL - DEBUG - global iter =   18 total obj: 21151.1106
2021-08-04 18:38:53,797 - DeepRL - DEBUG - global iter =   19 total obj: 21100.2303
2021-08-04 18:38:53,798 - DeepRL - DEBUG - started testing...
2021-08-04 18:38:58,611 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/taskData_staiTop/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-04 18:41:33,379 - DeepRL - DEBUG - global iter =   30 total obj: 20847.4743
2021-08-04 18:41:46,899 - DeepRL - DEBUG - global iter =   31 total obj: 20812.1255
2021-08-04 18:42:00,400 - DeepRL - DEBUG - global iter =   32 total obj: 20800.1829
2021-08-04 18:42:13,871 - DeepRL - DEBUG - global iter =   33 total obj: 20842.4427
2021-08-04 18:42:27,332 - DeepRL - DEBUG - global iter =   34 total obj: 20795.7479
2021-08-04 18:42:40,796 - DeepRL - DEBUG - global iter =   35 total obj: 20837.7703
2021-08-04 18:42:54,281 - DeepRL - DEBUG - global iter =   36 total obj: 20778.3925
2021-08-04 18:43:07,760 - DeepRL - DEBUG - global iter =   37 total obj: 20764.0064
2021-08-04 18:43:21,265 - DeepRL - DEBUG - global iter =   38 total obj: 20704.5516
2021-08-04 18:43:34,738 - DeepRL - DEBUG - global iter =   39 total obj: 20739.9540
2021-08-04 18:43:34,739 - DeepRL - DEBUG - started testing...
2021-08-04 18:43:40,162 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/taskData_staiTop/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-08-04 18:43:54,420 - DeepRL - DEBUG - global iter =   40 total obj: 20796.3276
2021-08-04 18:44:07,896 - DeepRL - DEBUG - global iter =   41 total obj: 20767.0707
2021-08-04 18:44:21,393 - DeepRL - DEBUG - global iter =   42 total obj: 20712.9613
2021-08-04 18:44:34,866 - DeepRL - DEBUG - global iter =   43 total obj: 20730.3841
2021-08-04 18:44:48,311 - DeepRL - DEBUG - global iter =   44 total obj: 20672.8542
2021-08-04 18:45:01,933 - DeepRL - DEBUG - global iter =   45 total obj: 20654.0730
2021-08-04 18:45:15,393 - DeepRL - DEBUG - global iter =   46 total obj: 20663.6501
2021-08-04 18:45:28,865 - DeepRL - DEBUG - global iter =   47 total obj: 20640.5048
2021-08-04 18:45:42,560 - DeepRL - DEBUG - global iter =   48 total obj: 20659.3115
2021-08-04 18:45:55,142 - DeepRL - DEBUG - global iter =   49 total obj: 20719.2290
2021-08-04 18:45:55,143 - DeepRL - DEBUG - opt finished.
2021-08-04 18:45:55,144 - DeepRL - DEBUG - started testing...
2021-08-04 18:46:00,676 - DeepRL - DEBUG 

INFO:tensorflow:../results/taskData_staiTop/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
test_data["S248"]

[{'action': array([[1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
          0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
          1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
          1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
          1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
          0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
          0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
          1, 0, 0, 1, 1, 1]]),
  'state': array([[[ 4,  7],
          [ 7,  8],
          [ 8,  7],
          [ 7,  4],
          [ 8,  4],
          [ 7,  8],
          [ 4,  7],
          [ 4,  7],
          [ 8,  7],
          [ 7,  4],
          [ 8,  7],
          [ 4,  8],
          [ 7,  8],
          [ 4,  8],
          [ 8,  4],
          [ 4,  8],
          [ 9,  4],
          [ 4,  9],
          [ 9,  4],
          [ 9,  4],
          [ 4,  9]